### Exercício 2 - Pedro Gonçalves (A82313) & Roberto Cachada (A81012)

Este exercício consiste na re-implementação do exercício anterior, mas utilizando ferramentas baseadas em Curvas Elípticas. Assim sendo, a cifra *AES (Advanced Encryption Standard)* será substituída pela *ChaCha20Poly1305*, o protocolo *Diffie-Hellman* pelo *Elliptic-Curve Diffie–Hellman* e o esquema de assinaturas *DSA (Digital Signature Algorithm)* pelo *Elliptic-Curve DSA*.

##### Alínea a)

- Para resolver esta primeira alínea foram criadas as funções ***genNounce***, ***mac***, ***encrypt*** e ***decrypt***-
- A função genNounce, tal como o nome sugere, serve para gerar *nounces*.
- A funçao ***mac*** serve para garantir a autenticação de texto limpo e também verificar a autenticidade do mesmo, utilizando para tal uma função de *hash* criptográfica juntamente com uma chave para cumprir esse objetivo.
- Já a função ***encrypt*** serve para encriptar/autenticar o texto limpo utilizando para tal a cifra *ChaCha20Poly1305*, que consiste na junção das cifra simétrica *ChaCha20* com o MAC *Poly1305*. Por fim a função ***decrypt*** serve para decifrar os resultados produzidos pela função ***encrypt***, verificando em simultâneo a sua autenticidade.

In [1]:
import os
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives import hashes, hmac

def genNounce(tamanho): #tamanho em bytes
    nounce = os.urandom(tamanho)
    return nounce

def mac(chave, textolimpo, tag=None):
    h = hmac.HMAC(chave,hashes.SHA256(),default_backend())
    h.update(textolimpo)
    if tag == None: #Se não existir tag, autentica o texto limpo
        return h.finalize()
    h.verify(tag) #Caso contrário verifica a autenticidade da tag

def encrypt(chave, textolimpo):
    chacha = ChaCha20Poly1305(chave) #Criação de um objeto ChaCha20Poly1305
    nounce = genNounce(12) #Geração de um nounce de 12 bytes
    cyphertext = chacha.encrypt(nounce, textolimpo,None) #Encripta o texto limpo
    return cyphertext, nounce

def decrypt(chave, nounce, cyphertext):
    chacha = ChaCha20Poly1305(chave) #Criação de um objeto ChaCha20Poly1305
    textolimpo = chacha.decrypt(nounce, cyphertext,None) #Retorna o texto limpo
    return textolimpo

##### Alínea b)

- Nesta alínea foi criada a função ***ecSTS***, onde está definido o protocolo de troca de chaves *Elliptic-Curve Diffie–Hellman* e a autenticação dos agentes atráves do esquema de assinaturas *Elliptic-Curve DSA*.
- Para tal, são geradas chaves públicas e privadas para ambos os protocolos. Em seguida é executado o processo de troca de chaves *ECDH*, sendo em simultâneo verificada a autenticidade dos agentes envolvidos através do esquema de assinaturas *ECDSA*.

In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization

salt = os.urandom(16) #Salt partilhado

def ecSTS(agente, conn):
    #--ECDH--
    chavePrivadaECDH = ec.generate_private_key(ec.SECP384R1(), default_backend())
    chavePublicaECDH = chavePrivadaECDH.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo) #Gera uma chave pública ECDH e serializa a mesma

    
    #ECDSA
    chavePrivadaECDSA = ec.generate_private_key(ec.SECP384R1(), default_backend())
    chavePublicaECDSA = chavePrivadaECDSA.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo) #Gera uma chave pública ECDSA e serializa a mesma

    assinatura = chavePrivadaECDSA.sign(chavePublicaECDH, ec.ECDSA(hashes.SHA256()))

    
    #É criada uma stream de dados(dicionário), que contém ambas as chaves públicas do agente e a sua assinatura ECDSA
    dados = {'CP_ECDH' : chavePublicaECDH, 'CP_ECDSA' : chavePublicaECDSA, 'Assinatura' : assinatura}
    conn.send(dados) #É enviada a stream de dados

    dados = conn.recv() #É recebida a stream de dados do outro agente
    peerECDH_PK_Bytes = dados['CP_ECDH']
    peerECDSA_PK_Bytes = dados['CP_ECDSA']
    peerSignature = dados['Assinatura']

    
    #Chaves recebidas são deserializadas
    peerECDH_PK = serialization.load_pem_public_key(peerECDH_PK_Bytes, backend=default_backend())
    peerECDSA_PK = serialization.load_pem_public_key(peerECDSA_PK_Bytes, backend=default_backend())

    try: #É verificada a validade da assinatura do agente oposto
        peerECDSA_PK.verify(peerSignature, peerECDH_PK_Bytes, ec.ECDSA(hashes.SHA256()))
        print(agente + ": Assinatura validada com sucesso")
    except:
        print(agente + ": Assinatura não validada")

    
    #Obtenção da chave ECDH partilhada
    chavePartilhada = chavePrivadaECDH.exchange(ec.ECDH(), peerECDH_PK)
    chaveDerivada = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    info=b'handshake data',
    backend=default_backend()
    ).derive(chavePartilhada)


    #Verificação se a chave obtida por ambos os agentes foi a mesma através de um Hmac
    hashVerificacao = mac(b'PasswordVerificacao',chaveDerivada)
    conn.send(hashVerificacao)
    peerHashChave = conn.recv()
    mac(b'PasswordVerificacao', chaveDerivada, peerHashChave)
    print(agente + ": Processo ECDH concluído com sucesso")

    return chaveDerivada

##### Comunicação Privada
* Para os agentes comunicarem entre si, são criados dois processos que comunicam entre si através de um *pipe*, permitindo assim ao ***Emitter***, após ter completado o processo de troca de chaves, enviar uma *stream* de informação encriptada ao ***Receiver*** fechando ambos a conexão após o envio/receção dessa *stream* de dados.

In [3]:
from multiprocessing import Pipe, Process

class PipeCom(object):
    def __init__(self,left,right,timeout=None): #Recebe os agentes e um timeout para os processos a serem criados
        left_end, right_end = Pipe() #São criadas as "pontas" do Pipe
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,)) #Cria um processo e define qual o agente desse processo
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end) #Define quais as "pontas" a que cada agente tem acesso
        self.right = lambda : right(right_end)
    
    def executa(self): #Cria os processos e executa os agentes
        self.lproc.start()
        self.rproc.start()  
        self.lproc.join(self.timeout)
        self.rproc.join(self.timeout)

In [4]:
mensagem = b"Mensagem a encriptar e enviar e caracteres aleatorios jkc kscskdjnc hfjshc kjfhskjcnkjs jkcvsjkhvnkj"

def Receiver(conn):
    chaveR = ecSTS('Receiver', conn)
    
    try:#Separar os diferentes componentes da stream(dicionário) de dados recebida
        data = conn.recv() #Recebe dados
        ciphertext = data['cipher']
        nounce = data['nounce']
        print(decrypt( #Decifra o ciphertext e verifica a autenticidade dos metadados
            chaveR,
            nounce,
            ciphertext
        ))
        print("Receiver: Processo concluído com sucesso")
    except:
        print("Receiver: Erro")
        
    conn.close() #Fecha a sua "ponta" do Pipe

def Emitter(conn):
    chaveE = ecSTS('Emitter', conn)
    
    try:
        ciphertext, nounce = encrypt( #Cifra texto limpo e autentica o mesmo
            chaveE,
            mensagem
        )
        data = {'cipher' : ciphertext , 'nounce' : nounce } #Organiza os elementos numa Stream(dicionário)
        conn.send(data) #Envia dados
        print("Emitter: Dados Enviados")
    except:
        print("Emitter: Erro")
        
    conn.close() #Fecha a sua "ponta" do Pipe

In [5]:
PipeCom(Emitter, Receiver).executa()

Emitter: Assinatura validada com sucesso
Receiver: Assinatura validada com sucesso
Emitter: Processo ECDH concluído com sucesso
Receiver: Processo ECDH concluído com sucesso
Emitter: Dados Enviados
b'Mensagem a encriptar e enviar e caracteres aleatorios jkc kscskdjnc hfjshc kjfhskjcnkjs jkcvsjkhvnkj'
Receiver: Processo concluído com sucesso
